In [5]:
import re
import time
import requests
import numpy as np

from bs4 import BeautifulSoup
from collections import Counter
from selenium import webdriver

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

from selenium.webdriver.common.by import By


In [6]:
url = 'https://www.dataanalyst.com/?ea8d9d2b_page='

urls = []
for i in range(1,8):
    urls.append(url+str(i))
    
print(urls[0])
range(len(urls))

https://www.dataanalyst.com/?ea8d9d2b_page=1


range(0, 7)

In [7]:
## launch headless Edge instance
options = webdriver.EdgeOptions()
options.add_argument('--headless')


In [8]:
## initiate variables for loop
df = pd.DataFrame(columns = ['job_title', 'job_url','job_country', 'job_company', 'job_location', 'job_workplace', 'job_industry', 'job_experience', 'job_salary', 'job_published', 'job_roles', 'job_requirements'])
i = 0
for ii in range(len(urls)): 
    
    print("Starting")
    print(ii)
    
    driver = webdriver.Edge(options = options)
    driver.get(urls[ii])

    ## grab html from page
    page = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(page, 'html.parser')

    job_rows = soup.find_all('a', class_ = 'card job cms-jobs-counter w-inline-block') 
    print("Number of jobs")
    print(len(job_rows))

    ## loop through rows, grabbing unique job posting url from each  
    for iii in range(len(job_rows)): # 
        
        ## grab url for unique job post
        job_link = re.search(r'(?<=href=\").*(?=\"><div class=\"html-embed-5 w-embed\")', str(job_rows[iii])).group()
        print(job_link)
        j_url = 'https://www.dataanalyst.com/'
        job_url = str(j_url + 
                      re.search(r'(?<=href=\").*(?=\"><div class=\"html-embed-5 w-embed\")', 
                                str(job_rows[iii])).group()
                     )

        # Lets try getting everything in a list
        a = re.findall(r'(?<=\>)[0-9a-zA-z\s\,\.\-]+(?=\<)',str(job_rows[iii]))

        # title 0, company 1
        job_title = a[0]
        job_company = a[1]
        # Country
        job_country = a[a.index("Country") + 1]
        # Location
        job_location = a[a.index("Location") + 1]
        # Workplace
        job_workplace = a[a.index("Workplace") + 1]
        # Industry
        job_industry = a[a.index("Industry") + 1]
        # published
        job_published = a[a.index("Published") + 1]

        # Still need experience, salary and currency
        # Experience
        a = re.findall(r'(?<=\>)[0-9]+.*[0-9]+.*(?=years)',str(job_rows[iii]))
        job_experience = a
        # Salary
        try:
            a = re.search(r'(?<= class=\"card-job-category-title-wrapper\"\>\<div\>Salary\<\/div\>\<\/div\>\<div class=\"card-job-category-text\">).*(?=\<\/div\>\<\/div\>\<div class=\"card-job-category-wrapper-industry\")', str(job_rows[iii])).group()
            job_salary = a
        except:
            job_salary = 0

        ## capture html from post page
        job_page = requests.get(job_url)
        job_soup = BeautifulSoup(job_page.content, 'html.parser')

        # grab job roles
        job_roles = job_soup.find('div', class_ = 'rich-text-block-3 w-richtext')
        job_roles = re.sub(r'\<.*\>', '', job_roles.prettify())

        # grab job responsibilities
        job_requirements = job_soup.find('div', class_ = 'rich-text-block-4 w-richtext')
        job_requirements = re.sub(r'\<.*\>', '', job_requirements.prettify())

        ## move all variables to dataframe
        df.loc[i, 'job_title'] = job_title
        df.loc[i,'job_url'] = job_url
        df.loc[i,'job_company'] = job_company
        df.loc[i,'job_country'] = job_country
        df.loc[i,'job_location'] = job_location
        df.loc[i,'job_workplace'] = job_workplace
        df.loc[i,'job_industry'] = job_industry
        df.loc[i,'job_experience'] = job_experience
        df.loc[i,'job_salary'] = job_salary
        df.loc[i,'job_published'] = job_published
        df.loc[i,'job_roles'] = job_roles
        df.loc[i,'job_requirements'] = job_requirements

        time.sleep(1)
        
        i = i + 1
        
    df2 = df
    df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
    df2.job_roles = df2.job_roles.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
    df2.job_requirements = df2.job_requirements.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
    df2.to_csv("data_analyst_dot_com.csv")
        
    time.sleep(2)

        
        

Starting
0
Number of jobs
100
/job/senior-data-analyst-7b
/job/data-analyst-579
/job/senior-data-analyst-1c9
/job/data-analyst-da4
/job/senior-data-analyst-db
/job/financial-data-analyst
/job/senior-data-analyst-healthcare
/job/data-analyst-1e7
/job/data-analyst-bi-0
/job/data-analyst-grants
/job/data-analyst-claims
/job/data-analyst-analytics-visualization
/job/data-analyst-098
/job/data-analyst-e9
/job/data-analyst-ec
/job/senior-data-analyst-0a8
/job/senior-data-analyst-d1
/job/data-analyst-marketing-4
/job/data-analyst-0e3
/job/data-analyst-product-analytics-5
/job/data-analyst-69
/job/data-analyst-product-analytics
/job/data-analyst-d6
/job/data-analyst-governance
/job/senior-data-analyst-f1
/job/data-analyst-4cc
/job/data-analyst-bi-4
/job/data-analyst-d4
/job/data-analyst-5d
/job/senior-data-analyst-a7
/job/data-analyst-39
/job/senior-data-analyst-25
/job/director-data-analyst
/job/data-analyst-business-3
/job/lead-data-analyst-growth
/job/senior-data-analyst-2f
/job/data-analys

C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:91: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:92: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_roles = df2.job_roles.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:93: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_requirements = df2.job_requirements.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')


Starting
1
Number of jobs
100
/job/senior-data-analyst-975
/job/data-analyst-accounting
/job/lead-data-analyst-0b
/job/data-analyst-epidemiologist
/job/data-analyst-84
/job/data-analyst-b55
/job/senior-data-analyst-c8
/job/senior-data-analyst-marketing-0
/job/data-analyst-254
/job/data-analyst-pricing
/job/data-analyst-6f
/job/data-analyst-recruiting
/job/senior-data-analyst-fraud
/job/senior-data-analyst-edd
/job/data-analyst-aa
/job/data-analyst-geospatial
/job/data-analyst-97
/job/data-analyst-ea8
/job/data-analyst-da8
/job/data-analyst-79
/job/senior-data-analyst-4bd
/job/senior-data-analyst-28
/job/senior-data-analyst-dcb
/job/senior-data-analyst-6f0
/job/senior-data-analyst-110
/job/senior-data-analyst-8d
/job/senior-data-analyst-9e
/job/data-analyst-risk
/job/data-analyst-7c8
/job/data-analyst-02b
/job/apprentice-data-analyst-3
/job/senior-data-analyst-8e
/job/senior-data-analyst-7c
/job/apprentice-data-analyst
/job/lead-data-analyst-7e
/job/data-analyst-2ff
/job/data-analyst-3b

C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:91: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:92: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_roles = df2.job_roles.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:93: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_requirements = df2.job_requirements.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')


Starting
2
Number of jobs
100
/job/data-analyst-games-4
/job/senior-data-analyst-customer-acq
/job/senior-data-analyst-434
/job/senior-data-analyst-93
/job/lead-data-analyst-7
/job/data-analyst-fleet
/job/data-analyst-3fc
/job/data-analyst-a9a
/job/lead-data-analyst-62
/job/master-data-analyst-3
/job/principal-data-analyst-e
/job/senior-data-analyst-6f2f
/job/data-analyst-f5
/job/data-analyst-cloud
/job/data-analyst-f4c
/job/data-analyst-55
/job/senior-data-analyst-4e8
/job/data-analyst-103
/job/senior-data-analyst-6f2
/job/senior-data-analyst-47
/job/data-analyst-a3
/job/data-analyst-0b
/job/data-analyst-finance-c
/job/data-analyst-finance-8
/job/data-analyst-f2a1
/job/data-analyst-09d
/job/data-analyst-ecology
/job/data-analyst-6e
/job/data-analyst-credit
/job/data-analyst-13
/job/senior-data-analyst-61
/job/senior-data-analyst-63c
/job/data-analyst-production
/job/data-analyst-ef8
/job/data-analyst-ef7
/job/data-analyst-9b
/job/data-analyst-68e
/job/data-analyst-c0
/job/senior-data-

...

C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:91: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:92: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_roles = df2.job_roles.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:93: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_requirements = df2.job_requirements.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')


Starting
5
Number of jobs
100
/job/senior-data-analyst-c3
/job/data-analyst-bc
/job/data-analyst-f4
/job/data-analyst-f2a
/job/data-analyst-investments
/job/senior-data-analyst-d7
/job/senior-data-analyst-ops
/job/data-analyst-5ac
/job/data-analyst-5e2
/job/data-analyst-365
/job/data-analyst-9a
/job/data-analyst-marketing-6
/job/data-analyst-onboarding
/job/senior-data-analyst-b0
/job/senior-data-analyst-4a
/job/senior-data-analyst-e6a
/job/data-analyst-cx
/job/data-analyst-42
/job/data-analyst-96
/job/data-analyst-75
/job/senior-data-analyst-c5
/job/lead-data-analyst-f
/job/data-analyst-6d
/job/data-analyst-601
/job/data-analyst-mid-level
/job/senior-data-analyst-59
/job/data-analyst-business
/job/senior-data-analyst-hr
/job/data-analyst-marketing-e
/job/senior-data-analyst-health
/job/data-analyst-5f
/job/senior-data-analyst-r-d
/job/senior-data-analyst-60
/job/data-analyst-a5
/job/data-analyst-marketing-a
/job/senior-data-analyst-dc
/job/data-analyst-36
/job/lead-data-analyst-resear

C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:91: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:92: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_roles = df2.job_roles.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:93: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_requirements = df2.job_requirements.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')


Starting
6
Number of jobs
100
/job/data-analyst-09
/job/data-analyst-9c1
/job/senior-data-analyst-cd
/job/data-analyst-bi
/job/senior-data-analyst-e9
/job/data-analyst-2b
/job/data-analyst-monetization
/job/data-analyst-c7
/job/master-data-analyst
/job/data-analyst-ea
/job/data-analyst-cd
/job/senior-data-analyst-07
/job/senior-data-analyst-e6
/job/senior-data-analyst-d3
/job/senior-data-analyst-bf
/job/data-analyst-ce
/job/senior-data-analyst-50
/job/senior-data-analyst-29
/job/data-analyst-441
/job/senior-data-analyst-4b
/job/senior-data-analyst-d
/job/associate-data-analyst
/job/data-analyst-fb
/job/data-analyst-c33
/job/junior-data-analyst-a
/job/senior-data-analyst-56
/job/data-analyst-8f
/job/data-analyst-57
/job/vp-lead-data-analyst
/job/data-analyst-8a
/job/data-analyst-d9
/job/data-analyst-f7
/job/data-analyst-ebf
/job/data-analyst-78
/job/senior-data-analyst-27
/job/senior-data-analyst-fd
/job/data-analyst-25a
/job/data-analyst-ff
/job/senior-data-analyst-65
/job/junior-data-

C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:91: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:92: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_roles = df2.job_roles.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_22216\722653960.py:93: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_requirements = df2.job_requirements.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
